#### Подключение библиотек

In [ ]:
import numpy as np
import cv2 as cv

import os
from pathlib import Path

from skimage.restoration import denoise_nl_means, estimate_sigma
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage import img_as_ubyte
from skimage.util import random_noise

#### Обработка видео (реализация Non-local means алгоритма)

In [ ]:
# Non-local means algorithm returning psnr measure for denoised video
def nonLocalMeans(filename, params, output_name):
    
    # Get required parametrs
    h1, template_window_size, search_window_size = params
    
    # Capture original (without noise) video
    orig_cap = cv.VideoCapture("in/taolu.avi")
    average_psnr = 0
    
    # Capture video
    cap = cv.VideoCapture(filename)

    # Get frames count
    n_frames = int(cap.get(cv.CAP_PROP_FRAME_COUNT))
    print(f"Frames count: {n_frames}")

    # Get width and height of video stream
    w = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    h = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    print(f"Video size: {w}×{h} pixels")

    # Get video stream fps
    fps = cap.get(cv.CAP_PROP_FPS)
    print(f"Video fps: {fps:.5} frames per second")
    
    # Define the codec for output video
    fourcc = cv.VideoWriter_fourcc(*"I420")

    # Set up output video
    out = cv.VideoWriter(output_name, fourcc, fps, (w, h))
    
    # Define non-local means parametrs
    patch_kw = dict(patch_size=template_window_size, patch_distance=search_window_size, fast_mode=True, multichannel=True)

    for i in range(n_frames):

        res, frame = cap.read()
        _, orig_frame = orig_cap.read()
        if not res:
            print(f"Frame {i+1}: end of stream")
            break

        sigma_est = np.mean(estimate_sigma(frame.astype("float"), multichannel=True))

        denoised_frame = denoise_nl_means(frame.astype("float"), h=h1*sigma_est, sigma=sigma_est, **patch_kw)
        denoised_frame = img_as_ubyte(denoised_frame / 255)
        
        average_psnr += psnr(orig_frame, denoised_frame)
        
        out.write(denoised_frame)

        print(f"Frame {i+1} was denoised")
    
    average_psnr /= n_frames
    
    cap.release()
    out.release()
    cv.destroyAllWindows()
    
    return average_psnr

#### Подбор параметров

In [ ]:
H1 = [0.35, 0.6, 0.8, 1.15]
TMPLT_SIZE = [5, 7]
SRCH_SIZE = [6, 10]

In [ ]:
best_psnr = [-1 for i in range(5)]
best_videos = ["" for i in range(5)]
best_params = [[] for i in range(5)]

index = 0

for h1 in H1:
    for template_size in TMPLT_SIZE:
        for search_size in SRCH_SIZE:

            input_name = "in/taolu_artificial_noise.avi"
            output_name = "out/taolu_denoised_example" + str(index) + ".avi"
            
            params = [h1, template_size, search_size]
                    
            video_psnr = nonLocalMeans(input_name, params, output_name)
            
            min_best = min(best_psnr)
            i_min = best_psnr.index(min_best)
            if video_psnr > min_best:
                best_psnr[i_min] = video_psnr
                if best_videos[i_min] != "":
                    os.remove(best_videos[i_min])
                    # time.sleep(3)
                best_videos[i_min] = output_name
                best_params[i_min] = params
            else:
                os.remove(output_name)
                # time.sleep(3)
            
            index += 1
            print(f"Index: {index}")

In [ ]:
print("Best videos and their parametrs:")
for i in range (len(best_videos)):
    print(f"File name: {best_videos[i]} | PSNR: {best_psnr[i]}")
    print(f" h1: {best_params[i][0]}, template_window_size: {best_params[i][1]}, search_window_size: {best_params[i][2]}")